In [4]:
library(tidyverse) 
library(janitor)
library(patchwork)
library(wordcloud)
library(RColorBrewer)
library(wordcloud2)

In [5]:
file_name = "../input/us-accidents/US_Accidents_Dec21_updated.csv"

# Data preparation and cleaning
1. Load the data 
2. Look at the data columns
3. Check for missing values

In [6]:
df=read.csv(file_name)

In [7]:
glimpse(df)

In [8]:
summary(df)

In [9]:
df = clean_names(df)

In [10]:
df %>%
    summarise(across(everything(), ~ (sum(is.na(.))/nrow(df)) *100  )) %>% 
    pivot_longer(cols = everything(), names_to = "columns", values_to = "na") %>% 
    arrange(desc(na)) %>%
    filter(na > 0) -> na_percentage
na_percentage
    

In [11]:
options(repr.plot.width = 20, repr.plot.height = 10)
na_percentage %>% 
    ggplot() + 
    geom_col(mapping = aes(x=reorder(columns, -na), y = na, fill=columns)) + 
    labs(x = "Columns with na", y="%", title ="NA percentage by column")+
    theme(text = element_text(size = 20))

There is a considerable number of  missing values for some of the columns. We will try to avoid this columns in the analysis.

# Exploratory analysis and visualization

The dataset contains 47 different columns, without considering the columns with missing values we have 39 columns to perform the analysis. We'll analyze just some of them.

1. Weather condition
2. Astronomical_Twilight
3. All the POI annotations (Amenity, Bump, Crossing, Give_Way, Junction, No_Exit, Railway, Roundabout, Station, Stop, Traffic_Calming, Traffic_Signal, Turning_Loop)
4. Description

A total of 16 columns will be analyzed.

## Weather condition

In [12]:
glimpse(df$weather_condition)

In [13]:
n_distinct(df$weather_condition)

This column has 128 different categorical values.

In [14]:
df %>% group_by(weather_condition) %>% summarize(count=n()) %>% arrange(-count)

Among some values there's some with an empty character, we'll count them as a missing value.

In [15]:
df$weather_condition[df$weather_condition==""] = NA

Let's take the occurences of each weather condition as a percentage but only those above 1%.

In [16]:
df %>% group_by(weather_condition) %>%
    summarize(percentage=((n()/nrow(df)) * 100)) %>%
    arrange(-percentage) %>%
    filter(percentage > 1) -> weather_conditions_per
weather_conditions_per

On the plot we see that most accidents occur on a fair weather with a considerable difference. On weathers with a compromised visibility and road condition (Rain, haze, fog, light snow) there's considerably few accidents.

In [17]:
options(repr.plot.width = 22, repr.plot.height = 10)
weather_conditions_per %>% ggplot() +
    geom_col(mapping = aes(x=reorder(weather_condition, -percentage), y = percentage, fill = weather_condition)) + 
    labs(x = "Weather condition", y="%", title ="Percentage of accidents by weather condition") +
    theme(text= element_text(size=18))    

## Astronomical twilight

In [18]:
glimpse(df$astronomical_twilight)

In [19]:
n_distinct(df$astronomical_twilight)

This column has 3 different categorical values.

In [20]:
df %>% 
    group_by(astronomical_twilight) %>% 
    summarise(count=n()) 

There's some values with just an empty character too, we will count them as missing values.

In [21]:
df$astronomical_twilight[df$astronomical_twilight==""] = NA

In [22]:
df %>% group_by(astronomical_twilight) %>% summarise(count=n())

Let's take the occurences of as a percentage.

In [23]:
df %>% 
    group_by(astronomical_twilight) %>% 
    summarise(percentage= ((n()/nrow(df))*100) ) -> astronomical_twilight_per
astronomical_twilight_per

On the plot we see that most accidents occur during the day, which makes sense, the flow of traffic is higher during the day than at night.

In [24]:
options(repr.plot.width = 20, repr.plot.height = 8)
astronomical_twilight_per %>% ggplot() +
    geom_col(mapping = aes(x=astronomical_twilight, y = percentage, fill=astronomical_twilight)) +
    labs(x = "Time of the day (astronomical twilight)", y="%", title ="Accidents by time of the day (astronomical twilight)") +
    theme(text = element_text(size=18))

## POI annotations


These 13 columns contain 2 different categorical values (True/False) indicating that a road element was nearby the occured accident.

Let's check the occurences of each column and it's combination by percentage and considering only the ones above 1%.

In [25]:
df %>% 
    group_by(amenity, bump, crossing, give_way, junction, no_exit, railway, roundabout, station, stop, traffic_calming, traffic_signal, turning_loop) %>%
    summarise(percentage = n()/nrow(df) *100 ) %>%
    arrange(-percentage) %>%
    filter(percentage > 1) -> accidents_per_roadelement
accidents_per_roadelement

Let's create a tibble with the elements name and their corresponding percentage to plot it.

In [26]:
acc_road_element_per <- tibble(c("None", "Junction", "Traffic signal", "Crossing and traffic signal", "Crossing", "Stop", "Station" ), 
       pull(accidents_per_roadelement, percentage), .name_repair = ~ c("road_elements", "percentage"))
acc_road_element_per

On the plot we can see that most accidents occur close to none of the road elements considered on the dataset followed by junctions , traffic signals and crossing with traffic signal.

In [27]:
options(repr.plot.width = 20, repr.plot.height = 8)
acc_road_element_per %>%
    ggplot() +
    geom_col(mapping = aes(x=reorder(road_elements, -percentage), y=percentage, fill = road_elements)) +
    labs(x = "Road element", y="%", title ="Accidents by nearby road element") +
    theme(text = element_text(size=18))

## Severity

The severity is the impact of the accident on traffic.

In [28]:
glimpse(df$severity)

In [29]:
n_distinct(df$severity)

There's 4 grades of severity, being 1 the lowest and 4 the heighest.

Let's calculate the ocurrences of each value by percentage.

In [30]:
df %>%
    group_by(severity) %>%
    summarise(percentage = n() / nrow(df) *100) -> severity_per
severity_per

On the plot wi can see that the most recurrent severity is 2 by a large difference.

In [31]:
options(repr.plot.width = 20, repr.plot.height = 8)
severity_per %>%
    ggplot() + 
    geom_col(mapping = aes(x=severity, y=percentage, fill=severity)) +
    labs(x = "Severity", y="%", title ="Accidents by severity") +
    theme(text = element_text(size=18))

### Severity and weather condition

Now, we will take the severity of the accidents and see how the weather conditions affect it.

Specifically we will see the distribution of each severity with the weather conditions.

In [32]:
df %>%
    group_by(weather_condition) %>%
    filter(severity==4) %>%
    summarise(count = n()) %>%
    ggplot() +
    geom_density(mapping = aes(x=count))+
    scale_x_log10() + 
    theme(text = element_text(size=18)) +
    labs(x = "Weather condition", y="Density", title ="Density of severity 4 acccidents by weather condition")-> weather_severity4_dis


In [33]:
df %>%
    group_by(weather_condition) %>%
    filter(severity==3) %>%
    summarise(count = n()) %>%
    ggplot() +
    geom_density(mapping = aes(x=count))+
    scale_x_log10() + 
    theme(text = element_text(size=18)) +
    labs(x = "Weather condition", y="Density", title ="Density of severity 3 acccidents by weather condition")-> weather_severity3_dis

In [34]:
df %>%
    group_by(weather_condition) %>%
    filter(severity==2) %>%
    summarise(count = n()) %>%
    ggplot() +
    geom_density(mapping = aes(x=count)) +
    scale_x_log10() + 
    theme(text = element_text(size=18))+
    labs(x = "Weather condition", y="Density", title ="Density of severity 2 acccidents by weather condition")-> weather_severity2_dis

In [35]:
df %>%
    group_by(weather_condition) %>%
    filter(severity==1) %>%
    summarise(count = n()) %>%
    ggplot() +
    geom_density(mapping = aes(x=count))+
    scale_x_log10() + 
    theme(text = element_text(size=18)) +
    labs(x = "Weather condition", y="Density", title ="Density of severity 1 acccidents by weather condition") -> weather_severity1_dis

The distributions are very similar there's many weather conditions with few accidents and few weather conditions with many accidents. The form of the curve is similar between 4, 3, and 1 but not so similar with 2. However, is not extremly different to say that there's a completely different fenomenon occurring with severity 2. 

In [36]:
options(repr.plot.width = 20, repr.plot.height = 8)
weather_severity4_dis + weather_severity3_dis + weather_severity2_dis + weather_severity1_dis

To understand better what's going on we have to see which weather conditions has more accidents for each severity for percentages higher than 1.

In [37]:
df %>%
    group_by(weather_condition) %>%
    filter(severity==4) %>%
    summarise(count = n()/nrow(.)*100) %>%
    filter(count>1)%>%
    arrange(-count) -> severity4_weather
severity4_weather

In [38]:
options(repr.plot.width = 20, repr.plot.height = 8)
severity4_weather %>%
    ggplot() +
    geom_col(mapping=aes(x=reorder(weather_condition, -count), y=count, fill=weather_condition))+ 
    theme(text = element_text(size=18)) +
    labs(x = "Weather condition", y="%", title ="Severity 4 acccidents by weather condition")-> severity4_weather_plot

In [39]:
df %>%
    group_by(weather_condition) %>%
    filter(severity==3) %>%
    summarise(count = n()/nrow(.)*100) %>%
    filter(count>1)%>%
    arrange(-count) -> severity3_weather
severity3_weather

In [40]:
options(repr.plot.width = 20, repr.plot.height = 8)
severity3_weather %>%
    ggplot() +
    geom_col(mapping=aes(x=reorder(weather_condition, -count), y=count, fill=weather_condition))+ 
    theme(text = element_text(size=18)) +
    labs(x = "Weather condition", y="%", title ="Severity 3 acccidents by weather condition") -> severity3_weather_plot

In [41]:
df %>%
    group_by(weather_condition) %>%
    filter(severity==2) %>%
    summarise(count = n()/nrow(.)*100) %>%
    filter(count>1)%>%
    arrange(-count) -> severity2_weather
severity2_weather

In [42]:
options(repr.plot.width = 20, repr.plot.height = 8)
severity2_weather %>%
    ggplot() +
    geom_col(mapping=aes(x=reorder(weather_condition, -count), y=count, fill=weather_condition))+ 
    theme(text = element_text(size=18)) +
    labs(x = "Weather condition", y="%", title ="Severity 2 acccidents by weather condition") -> severity2_weather_plot

In [43]:
df %>%
    group_by(weather_condition) %>%
    filter(severity==1) %>%
    summarise(count = n()/nrow(.)*100) %>%
    filter(count>1)%>%
    arrange(-count) -> severity1_weather
severity1_weather

In [44]:
severity1_weather %>%
    ggplot() +
    geom_col(mapping=aes(x=reorder(weather_condition, -count), y=count, fill=weather_condition))+ 
    theme(text = element_text(size=18))+
    labs(x = "Weather condition", y="%", title ="Severity 1 acccidents by weather condition")-> severity1_weather_plot

On the plot we can see the ocurrences for each severity by weather condition. With fair weather occur the most accidents of almost all severities. We can see that weathers with no compromised visibility or road condition are the most recurrent on all severities. Also, we can see that weathers with compromised visibility or road condition are not recurrent not even on highest severities.

In [45]:
options(repr.plot.width = 30, repr.plot.height = 16)

severity4_weather_plot + severity3_weather_plot +severity2_weather_plot +severity1_weather_plot

## Description wordcloud

We will take the description column to create a wordcloud.

In [ ]:
library(tm)
library(wordcloud)

Due to memory overflow on Kaggle notebook, we took a sample of the dataset to create the wordcloud.

In [47]:
corpus = Corpus(VectorSource(sample(df$description,size=floor(nrow(df)/128))))

In [ ]:
corpus = tm_map(corpus, PlainTextDocument)
corpus = tm_map(corpus, tolower)
 
#Removing Punctuation
corpus = tm_map(corpus, removePunctuation)


#Remove stopwords
corpus = tm_map(corpus, removeWords, stopwords("english"))
 
 
# Eliminate white spaces
corpus = tm_map(corpus, stripWhitespace)

In [49]:
DTM <- TermDocumentMatrix(corpus)
mat <- as.matrix(DTM)
f <- sort(rowSums(mat),decreasing=TRUE)
dat <- data.frame(word = names(f),freq=f)

We can see some intersting key words on the wordcloud, some of them are abbreviation. We can highlight:

* *pkwy*: Parkway
* *i95*: Interstate 95
* *blvd*: Boulevard
* *i10*: Interstate 10
* *hwy*: Highway
* *fwy*: Freeway
* *us101*: U.S. Route 101
* *rdexit*: Road exit
* *aveexit*: Avenue exit

In [50]:
set.seed(100)
wordcloud2(dat, size = 1.5) 

# Conclusions

The dataset contain 47 columns and 2845342 rows. 8 columns contained missing values, later we learned that 2 other columns contained only a empty character as value and we traeted them as missing values. We analyzed 16 columns avoiding some of the columns with missing values. The columns analyzed were:

1. Weather condition
2. Astronomical_Twilight
3. All the POI annotations (Amenity, Bump, Crossing, Give_Way, Junction, No_Exit, Railway, Roundabout, Station, Stop, Traffic_Calming, Traffic_Signal, Turning_Loop)
4. Description

From these columns we learned that:

* 76% of the accidents occur on fairly normal weather conditions (Fair, Mostly Cloudy, Cloudy, Partly Cloudy, Clear).
* 76% of the accident occur during the day
* Only 26% of the accidents occured close to any of the road elements considered.
* Junctions are the road element where occur more accidents with 9.9%
* Severity 2 is the most recurrent with 89% of the accidents. 
* Accidents with bad weather doesn't have a higher severity.
* Accidents are not more frequent on bad weather conditions.
* From the description wordcloud we learned that:
    * The  U.S. Route 101, Interstate 95 and Interstate 10 are involved in quite some accidents.
    * Parkways, Highways, Freeways, Road exits, Avenue exits are involved in quite some accidents.

# Limitations and future work
* Enrich the analysis by considering more columns like Start_Time, City, and Street.
* Imputate the missing values of the columns already analyzed considering data from other columns:
    * Weather condition: By checking the other weather variables like Precipitation
    * Astronomical_Twilight: By checking the time of the accident and its coordinates.
* Find out the most recurrent roads, cities and states.